In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import re, gensim
from googletrans import Translator

In [ ]:
# def normalize_text(text):
#     # Keep only words, alphabets, and numbers
#     # text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

#     text = text.replace('\u202f', ' ')
    
#     text = ''.join([char for char in text if char.isascii()])

#     text = re.sub(r'https://\S+', ' ', text)

#     text = re.sub(r'[/.\n\t]', ' ', text)
    
#     text = re.sub(r'www.\S+', ' ', text)
#     # Normalize phone numbers
#     text = re.sub(r'\b\d{2}-\d{3}-\d{4}\b', ' ', text)
#     # Normalize numeric sequences
#     text = re.sub(r'\b\d+\b', ' ', text)

#     return text

In [ ]:
def sent_to_words(sentences):
  for sentence in sentences:
    sentence = re.sub(r'https?://\S+|www\.\S+', '', sentence)
    sentence = ''.join([char for char in sentence if char.isascii()])
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
def list_of_sentence(text):
    sentences = sent_tokenize(text.lower())
    # preprocessing 
    los = list(sent_to_words(sentences))
    # los = []
    # for sen in sentences:
    #     sen = normalize_text(sen)
    #     tokensen = word_tokenize(sen)
    #     tokensen = [element for element in tokensen if element[0].isalpha()]
    #     los.append(tokensen)
    return los

In [ ]:
def write_list_of_lists_to_text(file_path, data, delimiter='\t'):
    with open(file_path, 'w') as file:
        for sublist in data:
            line = delimiter.join(sublist)
            file.write(line + '\n')

def import_text_to_list_of_lists(file_path, delimiter='\t'):
    result = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace and newlines
            sublist = line.split(delimiter)
            result.append(sublist)
    return result

In [ ]:
def page_contents(link, div_class = 'elementor-widget-container'):
    driver = webdriver.Safari()
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    div_elements = soup.find_all('div', class_=div_class)

    paragraphs = []
    for div in div_elements:
        div_paragraphs = div.find_all('p')
        for paragraph in div_paragraphs:
            paragraphs.append(paragraph.text.replace('\xa0', ''))

    driver.quit()
    return paragraphs

In [ ]:
def translate_text(text, target_language):

    translator = Translator()
    detected_lang = "th"
    translated_text = translator.translate(text, src=detected_lang, dest=target_language)
    return translated_text.text

In [ ]:
driver = webdriver.Safari()
driver.get('https://scbtechx.io/services-products/')
soup = BeautifulSoup(driver.page_source, 'html.parser')

sp_h2_content = soup.find_all('h2', class_='elementor-heading-title elementor-size-default')
sp_content = [text.text for text in sp_h2_content]

driver.quit()

In [ ]:
driver = webdriver.Safari()
driver.get('https://scbtechx.io/terms-of-use/')
soup = BeautifulSoup(driver.page_source, 'html.parser')

span_elements = soup.find_all('span', style='font-weight: 400;')
tou_content = [span.text.replace('\xa0', '') for span in span_elements]

driver.quit()

In [ ]:
driver = webdriver.Safari()
driver.get('https://scbtechx.io/news/')

# Define the wait duration (in seconds)
wait_duration = 5

while True:
    try:
        # Find the button element by its class
        load_more_button = driver.find_element(By.CLASS_NAME, "wpr-load-more-btn")
        # load_more_button = WebDriverWait(driver, wait_duration).until(
        #     EC.visibility_of_element_located((By.CLASS_NAME, "wpr-load-more-btn"))
        # )
        driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        load_more_button.click()
        print("Clicked 'Load More' button")
        sleep(wait_duration)

    except ElementNotInteractableException:
        break

soup = BeautifulSoup(driver.page_source, 'html.parser')
# Find the section tag with the specified class
section = soup.find("section", class_="wpr-grid elementor-clearfix grid-images-loaded")

# Find all <a> tags within the section
a_tags = section.find_all("a")
keep_links = [link.get('href') for link in a_tags]
news_lisks = [link for link in set(keep_links) if link.startswith('https://scbtechx.io/') and 'https://scbtechx.io/category/' not in link]

driver.quit()

In [ ]:
links = ['https://scbtechx.io/privacy-notice/customers/',
         'https://scbtechx.io/privacy-notice/non-customers/',
         'https://scbtechx.io/data-platform/',
         'https://scbtechx.io/ekyc/',
         'https://scbtechx.io/ekyc-innovestx/']

links += news_lisks
print(len(links))

pages_cons = []
for link in links:

    pages_cons.append(page_contents(link))
    
    sleep(10)

all_contents = pages_cons + [tou_content] + [sp_content]

In [ ]:
all_corpus = []
for c in all_contents:
    all_corpus.append(list_of_sentence(" ".join(c)))

los_all_corpus = [list_of_sentence for corpus in all_corpus for list_of_sentence in corpus]

In [ ]:
file_path = 'src/TechX_corpus_norm.txt'
# file_path = 'src/TechX_corpus_normTH.txt'
write_list_of_lists_to_text(file_path, los_all_corpus, delimiter='\t')
techx_los = import_text_to_list_of_lists(file_path, delimiter='\t')

In [ ]:
# # Check word freq
# from collections import defaultdict

# word_freq = defaultdict(int)
# for sublist in techx_los:
#     for word in sublist:
#         word_freq[word] += 1
# word_freq = dict(word_freq)

PointX

In [ ]:
driver = webdriver.Safari()
driver.get('https://www.pointx.scb/promotions/')

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")

# Find the <div> element with the specified class
div = soup.find("div", class_="special-promotion-carousel-container carousel-share")
# Find all <a> tags within the <div> element
a_tags = div.find_all("a")
links = [a.get("href") for a in a_tags]

driver.quit()

In [ ]:
links = list(set(links))
links

In [ ]:
def pointx_page_contents(link):
    driver = webdriver.Safari()
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    p_tags = soup.find_all("p")
    p_texts = [re.sub(r"[\n\t]"," ",tag.text )for tag in p_tags if tag.text.strip()]
    # Find all <li> tags and extract the text
    li_tags = soup.find_all("li")
    li_texts = [re.sub(r"[\n\t]"," ",tag.text ) for tag in li_tags if tag.text.strip()]

    driver.quit()
    return p_texts + li_texts

In [ ]:
pages_cons = []

for link in links:
    pages_cons.append(pointx_page_contents(link))
    sleep(10)

In [ ]:
all_corpus = []
for content in pages_cons:
    los_en = [translate_text(sentence,'en') for sentence in content]
    all_corpus.append(list_of_sentence(" ".join(los_en)))

los_all_corpus = [list_of_sentence for corpus in all_corpus for list_of_sentence in corpus]

In [ ]:
file_path = 'src/PointX_corpus_norm.txt'
write_list_of_lists_to_text(file_path, los_all_corpus, delimiter='\t')
pointx_los = import_text_to_list_of_lists(file_path, delimiter='\t')

SCB

In [74]:
driver = webdriver.Safari()
driver.get('https://www.scb.co.th/en/personal-banking/stories.html')
# Find the "Load more" button element

wait_duration = 5
i = 8
tip4u_button = driver.find_element(By.XPATH, '//a[@class="ga_menu_section" and @title="Tips for You"]')
tip4u_button.click()

while (i>0):
    try:
        # Find the button element by its class
        load_more_button = driver.find_element(By.XPATH, '//a[@class="btn-default mar-top ga_see_all_button" and @title="Load more"]')
        # driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        load_more_button.click()
        print("Clicked 'Load More' button")
        sleep(wait_duration)
        i -= 1
    except ElementNotInteractableException:
        break

div_element = driver.find_element(By.CSS_SELECTOR, "div.row.relative.clearfix")

# Find all the link elements within the div
link_elements = div_element.find_elements(By.TAG_NAME, "a")

# Extract the href attribute value from each link element
links = [link.get_attribute("href") for link in link_elements]

driver.quit()

Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button


In [76]:
len(links)

74